In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import random
from typing import List
from functools import partial
import pickle
import math
from math import comb
import re

pd.set_option("display.max_rows", 83)
pd.set_option("display.max_columns", 83)

In [2]:
from Card import Card
from Deck import Deck

from simulate import get_counts, get_counts_probabilities_average

In [3]:
counts = get_counts()
print(counts.attrs["n"])
counts_probabilities_average = get_counts_probabilities_average()
# counts_probabilities_average
# counts

2124000


In [4]:
total_cards = 24
num_ranks = 4 # number of each rank (eg. 4 Aces in a deck)
num_suits = 6 # number of each suit (eg. 6 Hearts in a deck)

In [5]:
def total_ways(n):
    return comb(total_cards, n)

def ways_rank(n: int, rank_num: int):
    if n - rank_num < 0:
        return 0
    return comb(num_ranks, rank_num) * comb(total_cards - num_ranks, n - rank_num)

def ways_two_ranks(n, rank_num_a, rank_num_b):
    if n - rank_num_a - rank_num_b < 0:
        return 0
    return comb(num_ranks, rank_num_a) * comb(num_ranks, rank_num_b) * comb(total_cards - 2 * num_ranks, n - rank_num_a - rank_num_b)

def ways_suit(n, suit_num):
    if n - suit_num < 0:
        return 0
    return comb(num_suits, suit_num) * comb(total_cards - num_suits, n - suit_num)

def ways_poker(n):
    if n - 5 < 0:
        return 0
    return comb(5, 5) * comb(total_cards - 5, n - 5)

def ways_five_ranks(n, ranks_nums):
    if n - sum(ranks_nums) < 0:
        return 0
    
    ways = 1

    for rank_num in ranks_nums:
        ways *= comb(num_ranks, rank_num)
    ways *= comb(total_cards - 5 * num_ranks, n - sum(ranks_nums))
    return ways

In [6]:
c = pd.DataFrame(0, index = list(range(2,19)), columns = [
    'is_high_card',
    'is_pair',
    'is_two_pair',
    'is_small_straight',
    'is_big_straight',
    'is_three',
    'is_full',
    'is_quad',
    'is_flush',
    'is_small_poker',
    'is_big_poker'
], dtype=float)

for n in c.index:
    c['is_high_card'][n] = (ways_rank(n, 4) + ways_rank(n, 3) + ways_rank(n, 2) + ways_rank(n, 1)) / total_ways(n)
    c['is_pair'][n]      = (ways_rank(n, 4) + ways_rank(n, 3) + ways_rank(n, 2)) / total_ways(n)
    c['is_three'][n]     = (ways_rank(n, 4) + ways_rank(n, 3)) / total_ways(n)
    c['is_quad'][n]      = (ways_rank(n, 4)) / total_ways(n)

    positive_ranks_nums_for_two_pair = [(a, b) for (a, b) in list(itertools.product(range(0,5), range(0,5))) if a >= 2 and b >= 2]
    c['is_two_pair'][n] = sum([ways_two_ranks(n, a, b) for (a, b) in positive_ranks_nums_for_two_pair]) / total_ways(n)

    positive_ranks_nums_for_full = [(a, b) for (a, b) in list(itertools.product(range(0,5), range(0,5))) if a >= 3 and b >= 2]
    c['is_full'][n] = sum([ways_two_ranks(n, a, b) for (a, b) in positive_ranks_nums_for_full]) / total_ways(n)

    c['is_flush'][n] = (ways_suit(n, 5) + ways_suit(n, 6)) / total_ways(n)

    c['is_small_poker'][n] = ways_poker(n) / total_ways(n)
    c['is_big_poker'][n] = ways_poker(n) / total_ways(n)


    positive_ranks_nums_for_straight = list(itertools.product(range(1,5), repeat=5))
    c['is_small_straight'][n] = sum([ways_five_ranks(n, ranks_nums) for ranks_nums in positive_ranks_nums_for_straight]) / total_ways(n)
    c['is_big_straight'][n] = sum([ways_five_ranks(n, ranks_nums) for ranks_nums in positive_ranks_nums_for_straight]) / total_ways(n)

    # print(c.loc[col, 'is_high_card'])
c

,is_high_card,is_pair,is_two_pair,is_small_straight,is_big_straight,is_three,is_full,is_quad,is_flush,is_small_poker,is_big_poker
2,0.311594,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.436759,0.061265,0.000000,0.000000,0.000000,0.001976,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.544043,0.114907,0.003388,0.000000,0.000000,0.007623,0.000000,0.000094,0.000000,0.000000,0.000000
5,0.635234,0.179277,0.014681,0.024092,0.024092,0.018351,0.000565,0.000471,0.000141,0.000024,0.000024
6,0.712027,0.251270,0.038010,0.087491,0.087491,0.035291,0.003016,0.001412,0.000810,0.000141,0.000141
7,0.776021,0.328063,0.076208,0.187874,0.187874,0.059289,0.009361,0.003294,0.002704,0.000494,0.000494
8,0.828722,0.407115,0.130378,0.311876,0.311876,0.090909,0.022039,0.006588,0.006865,0.001318,0.001318
9,0.871542,0.486166,0.199819,0.443520,0.443520,0.130435,0.043576,0.011858,0.014666,0.002964,0.002964
10,0.905797,0.563241,0.282223,0.569480,0.569480,0.177866,0.076212,0.019763,0.027772,0.005929,0.005929
11,0.932712,0.636646,0.374036,0.681012,0.681012,0.232919,0.121551,0.031056,0.048055,0.010870,0.010870


In [7]:
def draw_corr(df):
    df = df.abs()
    vmax = df.max().max()
    vmin = df.min().min()
    return df.style.background_gradient(cmap='PuBu', vmax=vmax, vmin=0, axis = None)

print((c - counts_probabilities_average).abs().mean().mean())

draw_corr(c - counts_probabilities_average)

5.607571667779323e-05


,is_high_card,is_pair,is_two_pair,is_small_straight,is_big_straight,is_three,is_full,is_quad,is_flush,is_small_poker,is_big_poker
2,0.000069,0.000069,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000027,0.000029,0.000000,0.000000,0.000000,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000018,0.000032,0.000001,0.000000,0.000000,0.000017,0.000000,0.000003,0.000000,0.000000,0.000000
5,0.000002,0.000020,0.000009,0.000022,0.000072,0.000005,0.000000,0.000012,0.000007,0.000003,0.000000
6,0.000057,0.000036,0.000038,0.000305,0.000146,0.000022,0.000003,0.000002,0.000002,0.000003,0.000001
7,0.000098,0.000053,0.000032,0.000075,0.000012,0.000137,0.000030,0.000014,0.000011,0.000010,0.000009
8,0.000063,0.000014,0.000019,0.000153,0.000037,0.000036,0.000006,0.000013,0.000005,0.000003,0.000016
9,0.000021,0.000055,0.000068,0.000165,0.000049,0.000017,0.000015,0.000052,0.000087,0.000016,0.000002
10,0.000057,0.000066,0.000074,0.000665,0.000237,0.000036,0.000001,0.000027,0.000013,0.000028,0.000044
11,0.000117,0.000027,0.000009,0.000535,0.000372,0.000088,0.000068,0.000002,0.000019,0.000008,0.000016


In [8]:
counts_probabilities_average

,is_high_card,is_pair,is_two_pair,is_small_straight,is_big_straight,is_three,is_full,is_quad,is_flush,is_small_poker,is_big_poker
2,0.311525,0.021808,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.436731,0.061294,0.000000,0.000000,0.000000,0.001975,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.544060,0.114875,0.003387,0.000000,0.000000,0.007640,0.000000,0.000091,0.000000,0.000000,0.000000
5,0.635232,0.179297,0.014690,0.024113,0.024020,0.018346,0.000565,0.000459,0.000149,0.000027,0.000023
6,0.712084,0.251234,0.037972,0.087187,0.087345,0.035269,0.003019,0.001413,0.000812,0.000144,0.000142
7,0.775923,0.328010,0.076176,0.187949,0.187886,0.059425,0.009391,0.003308,0.002693,0.000484,0.000485
8,0.828659,0.407128,0.130397,0.311723,0.311839,0.090945,0.022045,0.006600,0.006870,0.001320,0.001301
9,0.871562,0.486111,0.199751,0.443355,0.443471,0.130418,0.043561,0.011910,0.014753,0.002981,0.002966
10,0.905740,0.563307,0.282298,0.568815,0.569243,0.177830,0.076213,0.019790,0.027785,0.005957,0.005885
11,0.932829,0.636618,0.374028,0.681547,0.681384,0.232831,0.121483,0.031054,0.048074,0.010878,0.010886
